In [1]:
import torch
import clip
import laion_clap
import pandas as pd
print(pd.show_versions)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 28.0/28.0 [00:00<00:00, 98.5kB/s]
vocab.txt: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 232k/232k [00:00<00:00, 772kB/s]
tokenizer.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 466k/466k [00:00<00:00, 2.00MB/s]
config.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 570/570 [00:00<00:00, 1.20MB/s]
vocab.json: 100%|██████████████████████████████████████████████████████████

<function show_versions at 0x7fdfb0b597e0>


In [2]:
print(torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"
model1, preprocess1 = clip.load("ViT-B/32", device=device)

True


100%|███████████████████████████████████████| 338M/338M [04:45<00:00, 1.24MiB/s]


In [3]:
def process_and_add_embedding(csv_path, column_name, batch_size):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)

        # 新しい列 "clip_text_embed" を追加
        df["clip_text_embed"] = None

        # 指定された列のデータをリストとして取得
        text_data_list = df[column_name].tolist()

        # ミニバッチ処理
        for i in range(0, len(text_data_list), batch_size):
            batch_text_data = text_data_list[i:i+batch_size]

            try:
                # ここでテキストデータを512次元のテンソルに変換する処理を実装
                # 以下は仮の例としてランダムな512次元ベクトルにしています
                token = clip.tokenize(batch_text_data).to(device)
                embedding_tensor = model1.encode_text(token)

                # "clip_text_embed" 列にテンソルを格納
                df.loc[i:i+batch_size-1, "clip_text_embed"] = embedding_tensor.tolist()

            except Exception as e:
                # エラーが発生した場合、例外メッセージを格納
                df.loc[i:i+batch_size-1, "clip_text_embed"] = "ERROR"
                print(str(e))

        # 結果を表示
        print(df)

        # CSVファイルに結果を保存
        df.to_csv("../Datasets/archive/musiccaps-processed.csv", index=False)

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# 以下は使用例です
csv_file_path = "../Datasets/archive/musiccaps-public.csv"
target_column_name = "aspect_list"
batchsize = 8

process_and_add_embedding(csv_file_path, target_column_name, batchsize)


Input ['live performance', 'poor audio quality', 'ambient noises', 'school dance program', 'crowd noise', 'high gain volume', 'hissing tones', 'medium tempo', 'techno dance song', 'male singer', 'groovy bass line', 'techno dance rhythm', 'keyboard accompaniment', 'kids dancing', 'noisy crowds', 'home video', 'school program', 'noise'] is too long for context length 77
Input ['guitarist', 'male talking', 'twang sounds', 'mediocre audio quality', 'guitar string tones', 'guitar technique', 'fretboard sounds', 'guitar demo', 'online guitar lessons', 'home music video', 'youtube tutorial', 'bas]d audio quality', 'humming noise', 'energetic', 'passionate', 'spirited okay', 'strong voice', 'ambient room noises'] is too long for context length 77
Input ['male vocalist', 'medium fast tempo', 'monologue', 'background music', 'hit songs', 'retro', 'narrator', 'narration', 'documentary', 'animated singing', 'enthusiastic drumming', 'cymbal ride', 'electric guitar', 'groovy bass lines', 'rock and r

In [ ]:
def process_and_add_embedding(csv_path, column_name, batch_size=32):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)

        # 新しい列 "clip_text_embed" を追加
        df["clip_text_embed"] = None

        # 指定された列のデータをリストとして取得
        text_data_list = df[column_name].tolist()

        # バッチごとに処理
        for i in range(0, len(text_data_list), batch_size):
            batch_text_data = text_data_list[i:i+batch_size]
            batch_embeddings = []

            # バッチ内のテキストデータを処理
            for j, text_data in enumerate(batch_text_data):
                try:
                    token = clip.tokenize(text_data).to(device)
                    embedding_tensor = model1.encode_text(token)
                    batch_embeddings.append(embedding_tensor)

                    # "clip_text_embed" 列にテンソルを格納
                    df.at[i + j, "clip_text_embed"] = embedding_tensor

                except Exception as e:
                    # エラーが発生した場合、例外メッセージを格納
                    df.at[i + j, "clip_text_embed"] = "ERROR"
                    print(f"エラーが発生しました: {str(e)}")

            # メモリ解放
            del batch_embeddings

            # 進捗表示
            print(f"Processed {i + len(batch_text_data)} out of {len(text_data_list)}")

        # 結果を表示
        print(df)

        # CSVファイルに結果を保存
        df.to_csv("../Datasets/archive/musiccaps-processed.csv", index=False)

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# CSVファイル、抽出列を指定
csv_file_path = "../Datasets/archive/musiccaps-public.csv"
target_column_name = "aspect_list"

process_and_add_embedding(csv_file_path, target_column_name)


In [5]:
import pandas as pd

def process_and_add_embedding(csv_path, column_name):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)

        # 新しい列 "clip_text_embed" を追加
        df["clip_text_embed"] = None

        # 指定された列のデータをリストとして取得
        text_data_list = df[column_name].tolist()

        # テキストデータを順に処理して "clip_text_embed" に格納
        for i, text_data in enumerate(text_data_list[400:900]):
            try:
                
                torch.cuda.empty_cache()
                
                # ここでテキストデータを512次元のテンソルに変換する処理を実装
                # 以下は仮の例としてランダムな512次元ベクトルにしています
                token = clip.tokenize(text_data).to(device)
                embedding_tensor = model1.encode_text(token)
                
                # "clip_text_embed" 列にテンソルを格納
                df.at[i, "clip_text_embed"] = embedding_tensor
                
                #del token
                #del embedding_tensor
                #torch.cuda.empty_cache()
                
            except Exception as e:
                # エラーが発生した場合、例外メッセージを格納
                df.at[i, "clip_text_embed"] = "ERROR"
                print(str(e))

        # 結果を表示
        print(df)

        # CSVファイルに結果を保存
        df.to_csv("../Datasets/archive/musiccaps-processed.csv", index=False)

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# 以下は使用例です
csv_file_path = "../Datasets/archive/musiccaps-public.csv"
target_column_name = "aspect_list"

process_and_add_embedding(csv_file_path, target_column_name)

Input ['choral harmony', 'four part harmony', 'church choir', 'vocal harmony', 'western classical music', 'medium tempo', 'love song', 'heartfelt', 'gospel music', 'cathedral', 'meditational', 'devotional', 'prayer', 'composer', 'conductor', 'choir', 'hymns', 'praise and worship', 'church service', 'poor audio quality', 'heavenly', 'ethereal voices', 'christian choir'] is too long for context length 77
Input ['instrumental', 'fast tempo', 'rock', 'blues', 'trumpets', 'lively spirited', 'complex harmony', 'complex chords', 'cymbal rides', 'dance groove', 'peppy', 'perky', 'vivacious', 'keyboard harmony', 'piano accompaniment', 'groovy bass lines', 'acoustic guitar', 'electric guitar', 'slick drumming', 'trumpets', 'lively', 'ambient', 'soothing', 'improvisation', 'soul', 'funk/soul', 'rhythmic', 'dance groove', 'syncopated bass lines', 'rhythmic counter play', 'infectious drum grooves'] is too long for context length 77
Input ['instrumental', 'fast tempo', 'electronic music', 'dj', 'pun

In [7]:
def process_and_add_embedding(csv_path, column_name):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)

        # 新しい列 "clip_text_embed" を追加
        df["clip_text_embed"] = None

        # 指定された列のデータをリストとして取得
        text_data_list = df[column_name].tolist()

        # テキストデータを順に処理して "clip_text_embed" に格納
        for i, text_data in enumerate(text_data_list):
            try:
                with torch.no_grad():
                    token = clip.tokenize(text_data).to(device)
                    embedding_tensor = model1.encode_text(token)

                # "clip_text_embed" 列にテンソルを格納
                df.at[i, "clip_text_embed"] = embedding_tensor

                # CUDAメモリを解放
                torch.cuda.empty_cache()

            except Exception as e:
                # エラーが発生した場合、例外メッセージを格納
                df.at[i, "clip_text_embed"] = "ERROR"
                print(str(e))

        # 結果を表示
        print(df)

        # CSVファイルに結果を保存
        df.to_csv("../Datasets/archive/musiccaps-processed.csv", index=False)

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# CSVファイル、抽出列を指定
csv_file_path = "../Datasets/archive/musiccaps-public.csv"
target_column_name = "aspect_list"

process_and_add_embedding(csv_file_path, target_column_name)


Input ['live performance', 'poor audio quality', 'ambient noises', 'school dance program', 'crowd noise', 'high gain volume', 'hissing tones', 'medium tempo', 'techno dance song', 'male singer', 'groovy bass line', 'techno dance rhythm', 'keyboard accompaniment', 'kids dancing', 'noisy crowds', 'home video', 'school program', 'noise'] is too long for context length 77
Input ['guitarist', 'male talking', 'twang sounds', 'mediocre audio quality', 'guitar string tones', 'guitar technique', 'fretboard sounds', 'guitar demo', 'online guitar lessons', 'home music video', 'youtube tutorial', 'bas]d audio quality', 'humming noise', 'energetic', 'passionate', 'spirited okay', 'strong voice', 'ambient room noises'] is too long for context length 77
Input ['male vocalist', 'medium fast tempo', 'monologue', 'background music', 'hit songs', 'retro', 'narrator', 'narration', 'documentary', 'animated singing', 'enthusiastic drumming', 'cymbal ride', 'electric guitar', 'groovy bass lines', 'rock and r